In [5]:
import pandas as pd

albums_data = pd.read_csv("spotify_albums.csv")
artists_data = pd.read_csv("spotify_artists.csv")
tracks_data = pd.read_csv("spotify_tracks.csv")

In [6]:
## join artist genre information and album release date with track dataset
# drop irrelevant columns
# get only tracks after 1990
def join_genre_and_date(artist_df, album_df, track_df):
    album = album_df.rename(columns={'id':"album_id"}).set_index('album_id')
    artist = artist_df.rename(columns={'id':"artists_id",'name':"artists_name"}).set_index('artists_id')
    track = track_df.set_index('album_id').join(album['release_date'], on='album_id' )
    track.artists_id = track.artists_id.apply(lambda x: x[2:-2])
    track = track.set_index('artists_id').join(artist[['artists_name','genres']], on='artists_id' )
    track.reset_index(drop=False, inplace=True)
    track['release_year'] = pd.to_datetime(track.release_date).dt.year
    track.drop(columns = ['Unnamed: 0','country','track_name_prev','track_number','type'], inplace = True)
    
    return track[track.release_year >= 1990]

In [7]:
def get_filtered_track_df(df, genres_to_include):
    df['genres'] = df.genres.apply(lambda x: [i[1:-1] for i in str(x)[1:-1].split(", ")])
    df_exploded = df.explode("genres")[df.explode("genres")["genres"].isin(genres_to_include)]
    df_exploded.loc[df_exploded["genres"]=="korean pop", "genres"] = "k-pop"
    df_exploded_indices = list(df_exploded.index.unique())
    df = df[df.index.isin(df_exploded_indices)]
    df = df.reset_index(drop=True)
    return df

In [8]:
genres_to_include = genres = ['dance pop', 'electronic', 'electropop', 'hip hop', 'jazz', 'k-pop', 'latin', 'pop', 'pop rap', 'r&b', 'rock']
track_with_year_and_genre = join_genre_and_date(artists_data, albums_data, tracks_data)
filtered_track_df = get_filtered_track_df(track_with_year_and_genre, genres_to_include)

In [9]:
filtered_track_df["uri"] = filtered_track_df["uri"].str.replace("spotify:track:", "")
filtered_track_df = filtered_track_df.drop(columns=['analysis_url', 'available_markets'])

In [10]:
display(filtered_track_df.head())
filtered_track_df.columns

,artists_id,acousticness,danceability,disc_number,duration_ms,energy,href,id,instrumentalness,key,...,speechiness,tempo,time_signature,track_href,uri,valence,release_date,artists_name,genres,release_year
0,68WwJXWrpo1yVOOIZjLSeT,0.0268,0.506,1.0,248777.0,0.741,https://api.spotify.com/v1/tracks/0UATU9OJxh4m...,0UATU9OJxh4m3fwDljdGZn,0.000027,1.0,...,0.0349,94.042,4.0,https://api.spotify.com/v1/tracks/0UATU9OJxh4m...,0UATU9OJxh4m3fwDljdGZn,0.236,2018-09-28,Evalyn,"[electropop, indie electro-pop, indie poptimis...",2018
1,09xj0S68Y1OU1vHMCZAIvz,0.5050,0.487,1.0,171573.0,0.297,https://api.spotify.com/v1/tracks/4JH1M62gVDND...,4JH1M62gVDNDhDAUiQB3Qv,0.000052,11.0,...,0.0915,185.912,3.0,https://api.spotify.com/v1/tracks/4JH1M62gVDND...,4JH1M62gVDNDhDAUiQB3Qv,0.289,2001-08-21,Café Tacvba,"[latin, latin alternative, latin rock, mexican...",2001
2,6pSsE5y0uJMwYj83KrPyf9,0.1330,0.629,1.0,207396.0,0.706,https://api.spotify.com/v1/tracks/0h7Ld5CvgzaU...,0h7Ld5CvgzaUN1zA3tdyPq,0.000000,1.0,...,0.4360,81.220,4.0,https://api.spotify.com/v1/tracks/0h7Ld5CvgzaU...,0h7Ld5CvgzaUN1zA3tdyPq,0.543,2019-01-25,Dawn Richard,"[alternative r&b, deep pop r&b, escape room, h...",2019
3,7slfeZO9LsJbWgpkIoXBUJ,0.4060,0.590,1.0,279000.0,0.597,https://api.spotify.com/v1/tracks/4S1bYWrLOC8s...,4S1bYWrLOC8smuy8kJzxKQ,0.000023,9.0,...,0.0275,121.051,4.0,https://api.spotify.com/v1/tracks/4S1bYWrLOC8s...,4S1bYWrLOC8smuy8kJzxKQ,0.466,1995-09-12,Ricky Martin,"[dance pop, latin, latin pop, mexican pop, pop...",1995
4,09hVIj6vWgoCDtT03h8ZCa,0.0316,0.727,1.0,218773.0,0.380,https://api.spotify.com/v1/tracks/758mQT4zzlvB...,758mQT4zzlvBhy9PvNePwC,0.000000,7.0,...,0.3350,92.050,4.0,https://api.spotify.com/v1/tracks/758mQT4zzlvB...,758mQT4zzlvBhy9PvNePwC,0.455,1991-09-24,A Tribe Called Quest,"[alternative hip hop, conscious hip hop, east ...",1991


Index(['artists_id', 'acousticness', 'danceability', 'disc_number',
       'duration_ms', 'energy', 'href', 'id', 'instrumentalness', 'key',
       'liveness', 'loudness', 'lyrics', 'mode', 'name', 'playlist',
       'popularity', 'preview_url', 'speechiness', 'tempo', 'time_signature',
       'track_href', 'uri', 'valence', 'release_date', 'artists_name',
       'genres', 'release_year'],
      dtype='object')

In [11]:
filtered_track_df.to_csv("filtered_track_df.csv", index=False)